Attention! Restart the nookbook after running the cell below for the first time

In [ ]:
%%time 
#hide output of cell
%%capture 

import warnings;
warnings.filterwarnings('ignore');
!pip install tqdm
!pip install texthero;
!pip install bert-extractive-summarizer;
# !pip install spacy==2.1.3
# !pip install transformers==2.2.2
!pip install spacy;
!pip install transformers;
!pip install neuralcoref;
!python -m spacy download en_core_web_md;


ERROR: Operation cancelled by user

In [ ]:
%%time
%%capture

import pandas as pd;
import matplotlib.pyplot as plt;
import spacy;
from tqdm import tqdm
import texthero as hero;
# import fasttext;
!python -m spacy download en_core_web_md;


CPU times: user 16 ms, sys: 11.2 ms, total: 27.2 ms
Wall time: 4.14 s


# Classification

## FastText Facebook


In [ ]:
%cd '/content'
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!pip install .

/content
fatal: destination path 'fastText' already exists and is not an empty directory.
/content/fastText
Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3017117 sha256=c377660604dda492740ce1352f5a10ed3b521301e69c9f1c62757a4511ff432f
  Stored in directory: /tmp/pip-ephem-wheel-cache-s4ll7i78/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [ ]:
!cd cooking.stackexchange.txt;
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
!head cooking.stackexchange.txt;
!wc cooking.stackexchange.txt;
!head -n 12404 cooking.stackexchange.txt > cooking.train;
!tail -n 3000 cooking.stackexchange.txt > cooking.valid;


In [ ]:
model = fasttext.train_supervised(input="cooking.train");
model.save_model("model_cooking.bin");
model.predict("Which baking dish is best to bake a banana bread ?");
model.predict("Why not put knives in the dishwasher?");

In [ ]:
%%script false
!cat cooking.stackexchange.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > cooking.preprocessed.txt
!head -n 12404 cooking.preprocessed.txt > cooking.train
!tail -n 3000 cooking.preprocessed.txt > cooking.valid
# model = fasttext.train_supervised(input="cooking.train", lr=1.0)
model.test("cooking.valid")
# model = fasttext.train_supervised(input="cooking.train", lr=1.0, epoch=25, wordNgrams=2)
model = fasttext.train_supervised(input="cooking.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
model.predict("Which baking dish is best to bake a banana bread ?", k=-1, threshold=0.5)

## Bert Google

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
# !! uncomment for full data
# df = df[:100]
df.columns =['text','label']
# Computationally heavy, its best to limit rows first
lenRows = 1000 #43 seconds DistillBert - 1.5 minutes  BERT
# lenRows = len(df)

df = df[:lenRows]

In [ ]:
# Encoding the  the text columns
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
# Find the maximum len of tokens (sentence)
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# padding all sentence to the maximum sentence
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
%%time

""" No attention: """
## put sentences into a tensor with no attention
# input_ids = torch.tensor(np.array(padded))
## Putting the tensor into the disbert model
# with torch.no_grad():
    # last_hidden_states = model(input_ids)
# get the attention mask from where the padding is not zero or 1 
''' with attention: '''
attention_mask = np.where(padded != 0, 1, 0)
# input tnesor tensor
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)
# attention tensor
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
# features
features = last_hidden_states[0][:,0,:].numpy()
batch_1 = df[:lenRows]
# labels
labels = batch_1['label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
# Lostic regression for fitting 
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

CPU times: user 2min 49s, sys: 1.61 s, total: 2min 51s
Wall time: 1min 26s


In [ ]:
lr_clf.score(test_features, test_labels) 
# disbert .83 bert 0.804

0.804

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# disbert: .45 bert .49

Dummy classifier score: 0.492 (+/- 0.05)


Bert in one cell

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
df.columns =['text','label']
lenRows = 1000 #43 seconds DistillBert - 1.5 minutes  BERT
# lenRows = len(df)
df = df[:lenRows]
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
# input tnesor tensor
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)
# attention tensor
with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)
# features
features = last_hidden_states[0][:,0,:].numpy()
batch_1 = df[:lenRows]
# labels
labels = batch_1['label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
# Lostic regression for fitting 
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

lr_clf.score(test_features, test_labels) 

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Summerization

## Bert Summarizer

! Attention 1.3GB will be downloaded

In [ ]:
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler

handler = CoreferenceHandler(greedyness=.4)
# How coreference works:
# >>>handler.process('''My sister has a dog. She loves him.''', min_length=2)
# ['My sister has a dog.', 'My sister loves a dog.']

body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model = Summarizer(sentence_handler=handler)
model(body)
model(body2)


In [ ]:
from transformers import *

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
custom_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=custom_config)

# from summarizer import Summarizer

# body = 'Text body that you want to summarize with BERT'
# body2 = 'Something else you want to summarize with BERT'
# model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
# model(body)
# model(body2)


In [ ]:
from summarizer import Summarizer

body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

model = Summarizer()
result = model(body, min_length=60)
full = ''.join(result)
print(full)
"""
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. 
The building sold fairly quickly after being publicly placed on the market only two months ago.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.'
Still the building is among the best known in the city, even to people who have never been to New York.
"""


# Visualization

## Text Hero

In [ ]:
%%script false

import texthero as hero
df['clean_text'] = hero.clean(df['Complaint'])
df['tfidf_clean_text'] = hero.tfidf(df['clean_text'])
df['pca_tfidf_clean_text'] = hero.pca(df['tfidf_clean_text'])
hero.scatterplot(df, col='pca_tfidf_clean_text', color='Date', title="PCA BBC Sport news")
NUM_TOP_WORDS = 5
a = df.groupby('Date')['clean_text'].apply(lambda x: hero.top_words(x)[:NUM_TOP_WORDS])
a = pd.DataFrame(a)
a.to_excel('.//data//top_words.xlsx')
df['pca'] = (
   df['Complaint']
   .pipe(hero.clean)
   .pipe(hero.tfidf)
   .pipe(hero.pca)
)

hero.scatterplot(df, 'pca', color='Date', title="Complaints")
NUM_TOP_WORDS = 5
df.groupby('Date')['Complaint'].apply(lambda x: hero.top_words(x)[:NUM_TOP_WORDS])
df['tfidf'] = (
    df['Complaint']
    .pipe(hero.clean)
    .pipe(hero.tfidf)
)

df['kmeans_labels'] = (
    df['tfidf']
    .pipe(hero.kmeans, n_clusters=3)
    .astype(str)
)

df['pca'] = df['tfidf'].pipe(hero.pca)

p = hero.scatterplot(df, 'pca', color='kmeans_labels', title="K-means Clustering based on raw text")
p
df['clean_text'] = hero.clean(df['Complaint'])
df['tfidf_clean_text'] = hero.tfidf(df['clean_text'])
df['pca_tfidf_clean_text'] = hero.pca(df['tfidf_clean_text'])

# hero.scatterplot(df, col='pca_tfidf_clean_text', color='Date', title="PCA BBC Sport news")


df['kmeans_labels'] = (
    df['tfidf_clean_text']
    .pipe(hero.kmeans, n_clusters=2)
    .astype(str)
)

hero.scatterplot(df, 'pca_tfidf_clean_text', color='kmeans_labels', title="K-means Clustering based on clean text")
for j in range((int(max(df.kmeans_labels))+1)):
  print(str(j)+' =======')
  print(df[df.kmeans_labels ==str(j)].Complaint)
df[df.kmeans_labels =='6']
for j in range((int(max(df.kmeans_labels))+1)):
  print(str(j)+' =======')
  print(df[df.kmeans_labels ==str(j)].Complaint)

# References



1.   List item
2.   http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/

